In [ ]:
import cv2
import numpy as np

def detect_dominant_colors(image_path, mask_path, k=3):
    # Load the image and the mask
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load mask as grayscale
    
    # Ensure the mask is binary (0 and 255 values)
    _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    
    # Apply the mask to the image
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    
    # Reshape the image to a 2D array of pixels (excluding masked areas)
    pixels = masked_image.reshape(-1, 3)
    
    # Remove black pixels (masked-out areas where RGB = [0, 0, 0])
    pixels = pixels[np.any(pixels != [0, 0, 0], axis=1)]
    
    # Convert pixels to float32 for KMeans input (required by OpenCV)
    pixels = np.float32(pixels)
    
    # Define criteria for K-means (iterations and accuracy)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    
    # Run KMeans clustering on the pixels
    _, labels, centers = cv2.kmeans(
        pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS
    )
    
    # Convert the centers (dominant colors) back to integer type (RGB)
    dominant_colors = np.uint8(centers)
    
    return dominant_colors

# Example usage:
image_path = './downloads/Kit_left_arm_2.png'
mask_path = './downloads/Kit_left_arm_mask.png'  # Path to your mask image

dominant_colors = detect_dominant_colors(image_path, mask_path, k=3)

print("Dominant colors in the masked area are:", dominant_colors)

In [16]:
from PIL import Image

def fill_in_background_color(file):
    # Define the new background color (R, G, B)
    background_color = (255,255,255)
    
    # Open the existing image
    existing_image = Image.open(file)

    # Get the size of the existing image
    width, height = existing_image.size

    # Create a new image with the same size and the background color
    background = Image.new('RGB', (width, height), background_color)

    # Paste the existing image onto the background
    # We use the alpha channel if the existing image has transparency
    background.paste(existing_image, (0, 0), existing_image.convert('RGBA'))

    # Save the new image
    background.save(file)

fill_in_background_color('./downloads/Kit_right_arm_mask.png')

In [7]:
def detect_edges(image_path):
    # Load image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Apply Gaussian blur to reduce noise
    blurred_image = cv2.GaussianBlur(image, (5, 5), 0)
    
    # Perform Canny edge detection
    edges = cv2.Canny(blurred_image, 100, 200)
    
    return edges

# Example usage
edges = detect_edges(image_path)

# Display edges
cv2.imshow("Edges", edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
import cv2
import numpy as np

def process_image_white_to_black_transparent_to_white(image_path, save_path):
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)  # Load with alpha channel if exists

    # Check if image has 4 channels (RGBA)
    if image.shape[2] == 4:
        # Split into BGR and Alpha channels
        bgr_image = image[:, :, :3]  # BGR part of the image
        alpha_channel = image[:, :, 3]  # Alpha channel

        # Create a mask for fully transparent pixels (alpha == 0)
        transparent_mask = alpha_channel == 0

        # Set transparent pixels to white in the BGR image
        bgr_image[transparent_mask] = [255, 255, 255]

        # Now check for white pixels in the BGR image (RGB [255, 255, 255])
        white_mask = np.all(bgr_image == [255, 255, 255], axis=-1)

        # Set white pixels to black (RGB [0, 0, 0])
        bgr_image[white_mask] = [0, 0, 0]

        # Merge the processed BGR image with the original alpha channel (if needed)
        final_image = np.dstack((bgr_image, alpha_channel))

    else:
        # If there is no alpha channel, treat it as a simple BGR image
        bgr_image = image

        # Find white pixels in the BGR image (RGB [255, 255, 255])
        white_mask = np.all(bgr_image == [255, 255, 255], axis=-1)

        # Set white pixels to black (RGB [0, 0, 0])
        bgr_image[white_mask] = [0, 0, 0]

        # Set the image to white where no alpha (if no transparency is involved, this step is skipped)
        final_image = bgr_image

    # Save the processed image
    cv2.imwrite(save_path, final_image)

    return final_image

# Example usage:
image_path = './downloads/Kit_right_arm.png'
save_path = './downloads/Kit_right_arm_mask.png'

processed_image = process_image_white_to_black_transparent_to_white(image_path, save_path)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def detect_dominant_colors_cv2(image_path, mask_path, k=3):
    # Load the image and the mask
    image = cv2.imread(image_path)  # Image is loaded in BGR format
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load mask as grayscale
    
    # Ensure the mask is binary (0 and 255 values)
    _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    
    # Apply the mask to the image
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    
    # Convert BGR to RGB for color consistency
    masked_image_rgb = cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB)
    
    # Reshape the image to a 2D array of pixels (excluding masked areas)
    pixels = masked_image_rgb.reshape(-1, 3)
    
    # Remove black pixels (masked-out areas where RGB = [0, 0, 0])
    pixels = pixels[np.any(pixels != [0, 0, 0], axis=1)]
    
    # Check if there are enough pixels left for K-means
    if len(pixels) == 0:
        raise ValueError("No valid pixels found after applying the mask.")
    
    if len(pixels) < k:
        raise ValueError(f"Not enough pixels for {k} clusters. Found {len(pixels)} pixels.")

    # Convert pixels to float32 for KMeans input (required by OpenCV)
    pixels = np.float32(pixels)
    
    # Define criteria for K-means (iterations and accuracy)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    
    # Run KMeans clustering on the pixels
    _, labels, centers = cv2.kmeans(
        pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS
    )
    
    # Convert the centers (dominant colors) back to integer type (RGB)
    dominant_colors = np.uint8(centers)
    
    return dominant_colors, masked_image_rgb

image_path = './downloads/Kit_left_arm_2.png'
mask_path = './downloads/Kit_left_arm_mask.png'  # Path to your mask image

try:
    dominant_colors, masked_image_rgb = detect_dominant_colors_cv2(image_path, mask_path, k=3)

    # Display the masked image to visually check if it's correct
    plot_image(masked_image_rgb, "Masked Image")

    print("Dominant colors in the masked area are:", dominant_colors)

except ValueError as e:
    print("Error:", e)

# Function to display the dominant colors
def plot_dominant_colors(dominant_colors):
    plt.figure(figsize=(8, 2))
    plt.axis("off")
    color_rects = np.zeros((50, 50 * len(dominant_colors), 3), dtype=np.uint8)
    for i, color in enumerate(dominant_colors):
        color_rects[:, i * 50:(i + 1) * 50] = color
    plt.imshow(color_rects)
    plt.show()

# Plot the dominant colors
plot_dominant_colors(dominant_colors)